In [1]:
import csv
from src.session import get_assumed_session

aws_role = "data-science-role-prod"
access_key_id = ""
secret_access_key = ""
with open('keys/semidan_accessKeys.csv', 'r', encoding='utf-8-sig') as file:
    reader = csv.DictReader(file)
    for row in reader:
        access_key_id = row['Access key ID']
        secret_access_key = row['Secret access key']

# Assume the role and get a session
role_arn = f"arn:aws:iam::051581800907:role/{aws_role}"
role_session_name = "AssumeRoleSession"
region_name = "us-west-2"

# Change file path and s3_key but keep the same bucket name
assumed_session = get_assumed_session(role_arn, role_session_name, region_name, access_key_id, secret_access_key)
s3_client = assumed_session.client('s3')

Assumed role successfully.


In [2]:
# List all files in the bucket
bucket_name = 'newatlantis-kg-data-prod'

response = s3_client.list_objects_v2(Bucket=bucket_name)
if 'Contents' in response:
    for obj in response['Contents']:
        print(obj['Key'])
else:
    print("No files found in the bucket.")

Microbiomics_BGC_dataset_test.csv
compounds.json
export.cypher
export_data.csv
jolespin_table_upload/OceanDNA.metadata.bgc.kg.tsv.gz
jolespin_table_upload/OceanDNA.metadata.biosamples.kg-placeholder.tsv.gz
jolespin_table_upload/OceanDNA.metadata.genomes.kg.tsv
jolespin_table_upload/OceanDNA.metadata.proteins.kg.tsv.gz
modelSEED/compound_distances.csv.gz
modelSEED/compound_distances.parquet
modelSEED/compound_similarities.parquet
modelSEED/filtered_compounds.json
modelSEED/filtered_reactions.json
oceandna_bgc_catalog.tsv
oceandna_mags_taxonomy.tsv
oceandna_samples_metadata.tsv
reactions.json
relationships and properties.csv


## Check EC number annotations in protein data

In [4]:
import boto3
import gzip
import io

def read_first_n_lines(s3_client, bucket_name, file_key, n=10):
    # Create S3 client using the provided session
    s3 = s3_client
    
    # Get the object from S3
    obj = s3.get_object(Bucket=bucket_name, Key=file_key)
    
    # Read the content of the file
    with gzip.GzipFile(fileobj=io.BytesIO(obj['Body'].read())) as gzipfile:
        lines = []
        for _ in range(n):
            line = gzipfile.readline().decode('utf-8').strip()
            if not line:
                break
            lines.append(line)
    
    return lines



bucket_name = 'newatlantis-kg-data-prod'
file_key = 'jolespin_table_upload/OceanDNA.metadata.proteins.kg.tsv.gz'
n = 10

first_n_lines = read_first_n_lines(s3_client, bucket_name, file_key, n)

for i, line in enumerate(first_n_lines, 1):
    print(f"Line {i}: {line}")

Line 1: protein_id	contig_id	genome_id	kegg_ortholog_id	name	enzymes
Line 2: OceanDNA-b36493_00101_4	OceanDNA-b36493_00101	OceanDNA-b36493	K00001	alcohol dehydrogenase [EC:1.1.1.1]	{'1.1.1.1'}
Line 3: OceanDNA-b36526_00005_18	OceanDNA-b36526_00005	OceanDNA-b36526	K00001	alcohol dehydrogenase [EC:1.1.1.1]	{'1.1.1.1'}
Line 4: OceanDNA-b22918_00036_5	OceanDNA-b22918_00036	OceanDNA-b22918	K00001	alcohol dehydrogenase [EC:1.1.1.1]	{'1.1.1.1'}
Line 5: OceanDNA-b32316_00111_4	OceanDNA-b32316_00111	OceanDNA-b32316	K00001	alcohol dehydrogenase [EC:1.1.1.1]	{'1.1.1.1'}
Line 6: OceanDNA-b8055_00095_3	OceanDNA-b8055_00095	OceanDNA-b8055	K00001	alcohol dehydrogenase [EC:1.1.1.1]	{'1.1.1.1'}
Line 7: OceanDNA-b38295_00203_5	OceanDNA-b38295_00203	OceanDNA-b38295	K00001	alcohol dehydrogenase [EC:1.1.1.1]	{'1.1.1.1'}
Line 8: OceanDNA-b11251_00002_6	OceanDNA-b11251_00002	OceanDNA-b11251	K00001	alcohol dehydrogenase [EC:1.1.1.1]	{'1.1.1.1'}
Line 9: OceanDNA-b32434_00004_113	OceanDNA-b32434_00004	OceanDNA-

## Try to find proteins with more than one assigned ec number

In [5]:
import boto3
import gzip
import io
import ast

def find_first_multiple_ec(s3_client, bucket_name, file_key):
    s3 = s3_client
    obj = s3.get_object(Bucket=bucket_name, Key=file_key)
    
    with gzip.GzipFile(fileobj=io.BytesIO(obj['Body'].read())) as gzipfile:
        # Skip the header
        next(gzipfile)
        
        for line_number, line in enumerate(gzipfile, 2):  # Start from 2 to account for header
            decoded_line = line.decode('utf-8').strip()
            fields = decoded_line.split('\t')
            
            if len(fields) >= 6:  # Ensure we have enough fields
                enzymes_str = fields[5]
                try:
                    enzymes_set = ast.literal_eval(enzymes_str)
                    if isinstance(enzymes_set, set) and len(enzymes_set) > 1:
                        return line_number, decoded_line
                except (ValueError, SyntaxError):
                    # If we can't evaluate the string as a set, skip this line
                    continue
    
    return None, None  # If no matching row is found


bucket_name = 'newatlantis-kg-data-prod'
file_key = 'jolespin_table_upload/OceanDNA.metadata.proteins.kg.tsv.gz'

line_number, line = find_first_multiple_ec(s3_client, bucket_name, file_key)

if line_number:
    print(f"First row with multiple EC numbers (line {line_number}):")
    print(line)
else:
    print("No row with multiple EC numbers found.")

First row with multiple EC numbers (line 303):
OceanDNA-b40955_00050_5	OceanDNA-b40955_00050	OceanDNA-b40955	K00004	(R,R)-butanediol dehydrogenase / meso-butanediol dehydrogenase / diacetyl reductase [EC:1.1.1.4 1.1.1.- 1.1.1.303]	{'1.1.1.4', '1.1.1.-', '1.1.1.303'}


## Retrieve proteins sequences from IDs

In [ ]:
from src.databases import group_by_mag, retrieve_sequences

# protein_ids = read_protein_ids(protein_ids_file)
faa_dir = "/home/ec2-user/SageMaker/efs/sandbox/sandbox/development/jolespin/EC2_WorkingDirectory/science/Databases/NewAtlantisPlanktonic/Sources/OceanDNA/Genomes/Prokaryotic/"
output_file = "example.faa"
protein_ids = ["OceanDNA-b45352_00001_1", "OceanDNA-b22900_00165_4"]

mag_groups = group_by_mag(protein_ids)

retrieve_sequences(faa_dir, mag_groups, output_file)